# Was wordt lijst

Hoofdvraag: welke landelijke typologie (SBB-code, VvN of rVvN) wordt onze standaard die we als input voor Veg2Hab gebruiken en waarom?
- rVvN is het nieuwst, en beschrijft als het goed is de meeste vegetatietypen. gecheckt: als iets een SBB of VvN code heeft, heeft het ook altijd een rVvN code. Andersom niet.
- De opdrachtgever gaf als commentaar terug dat zij vinden dat VvN de standaard moet zijn.

## Was-wordt lijst inladen

In [175]:
import pandas as pd
import re
import numpy as np

In [176]:
ww = pd.read_excel('../data/5. Was-wordt-lijst-vegetatietypen-en-habitattypen-09-02-2021.xlsx')
ww = ww[['rVvN','VvN','SBB-code', 'Habitattypen (blauw: geldige aanvulling; rood: voorstel; zie Toelichting)']].rename(columns={'Habitattypen (blauw: geldige aanvulling; rood: voorstel; zie Toelichting)':'Habtype', 'SBB-code':'SBB'})

/home/jakko/git/veg2hab/.venv/lib/python3.8/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [177]:
ww.shape

(1244, 4)

In [178]:
# Verwijderen van rijen waarbij zowel VvN en SBB missen
ww = ww[~(ww['SBB'].isna() & ww['VvN'].isna())]

In [179]:
ww.shape

(1116, 4)

## Volledigheid typologie was-wordt

rVvN lijkt de meest specifieke vegetatietypologie. Alles met een waarde voor SBB of VvN heeft altijd (1 uitzondering, zie beneden) ook een rVvN code, andersom niet. 

Velden met een SBB code hebben altijd ook een habitattype ingevuld, velden met VvN of rVvN niet.

In [180]:
ww.isna().sum()

rVvN         1
VvN        129
SBB        224
Habtype    183
dtype: int64

In [181]:
# Rijen met een waarde voor rVvN hebben NIET altijd een waarde voor VvN of SBB
ww[~ww['rVvN'].isna()].isna().sum()

rVvN         0
VvN        128
SBB        224
Habtype    183
dtype: int64

In [182]:
# Rijen met een waarde voor VvN hebben WEL altijd een waarde voor rVvN, maar niet voor SBB
ww[~ww['VvN'].isna()].isna().sum()

rVvN         0
VvN          0
SBB        224
Habtype    183
dtype: int64

In [183]:
# Rijen met een waarde voor SBB-code hebben WEL (bijna) altijd een waarde voor rVvN, maar niet voor VvN
ww[~ww['SBB'].isna()].isna().sum()

rVvN         1
VvN        129
SBB          0
Habtype      0
dtype: int64

In [184]:
ww[~ww['SBB'].isna() & ww['rVvN'].isna()]

,rVvN,VvN,SBB,Habtype
1216,NaN,NaN,41A3d,H0000


In [185]:
ww[ww.duplicated(keep=False)]

,rVvN,VvN,SBB,Habtype


#### SBB naar VvN vertalen en vice versa

**Vraag:** Is er een 1-op-1 of veel-op-1 relatie tussen SBB en VvN?

**Antwoord:** Nee, maar wel bij benadering. 

SBB vertalen naar VvN levert in 612 gevallen een unieke match, in 111 gevallen geen code, en in 48 gevallen meerdere VvN codes.

VvN vertalen naar SBB levert in 501 gevallen een unieke macth, in 217 gevallen geen code, en in 105 gevallen meerdere SBB codes.

In [186]:
number_of_VvN_per_SBB = ww.groupby('SBB')['VvN'].nunique()

In [187]:
number_of_VvN_per_SBB.value_counts()

VvN
1    612
0    111
3     23
2     12
4     11
5      2
Name: count, dtype: int64

In [188]:
number_of_VvN_per_SBB[number_of_VvN_per_SBB == 5]

SBB
07A-a    5
33A5     5
Name: VvN, dtype: int64

In [189]:
number_of_SBB_per_VvN = ww.groupby('VvN')['SBB'].nunique()

In [190]:
number_of_SBB_per_VvN.value_counts()

SBB
1    501
0    217
2     72
3     24
5      4
4      4
6      1
Name: count, dtype: int64

In [191]:
number_of_SBB_per_VvN[number_of_SBB_per_VvN == 5]

VvN
08Bd01    5
11Aa02    5
16aB01    5
43Ab1d    5
Name: SBB, dtype: int64

### VvN rijen splitsen

Sommige rijen voor VvN hebben dubbele codes, soms met `( )` haakjes.

In [192]:
ww.shape

(1116, 4)

In [193]:
ww['VvN'] = ww['VvN'].str.split(',')
ww = ww.explode('VvN')

In [194]:
ww.shape

(1132, 4)

In [195]:
ww['VvN'][~ww['VvN'].isna()] = ww['VvN'][~ww['VvN'].isna()].apply(lambda x: re.sub('[()]', '', str(x)))

In [196]:
ww.to_excel('../data/ww_split.xlsx', index=False, header=True)